In [ ]:
pip install --user mediapipe opencv-python


In [5]:
import cv2
import mediapipe as mp

def detect_fingers_landmarks(landmarks):
    finger_tips = [4, 8, 12, 16, 20]  
    finger_letters = ['T', 'I', 'M', 'R', 'B']

    for tip, letter in zip(finger_tips, finger_letters):
        if tip == 4:  
            if landmarks[tip].x < landmarks[tip + 1].x:
                x, y = int(landmarks[tip].x * width), int(landmarks[tip].y * height)
                cv2.putText(frame, letter, (x - 10, y + 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)  # Red color: (B, G, R) = (0, 0, 255)
        elif landmarks[tip].y < landmarks[tip - 1].y:
            x, y = int(landmarks[tip].x * width), int(landmarks[tip].y * height)
            cv2.putText(frame, letter, (x - 10, y + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)  # Red color: (B, G, R) = (0, 0, 255)


mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

cap = cv2.VideoCapture(0)  
width, height = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for landmarks in results.multi_hand_landmarks:

            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

            detect_fingers_landmarks(landmarks.landmark)

    cv2.rectangle(frame, (20, 430), (250, 480), (0, 0, 0), -1)
    cv2.putText(frame, 'Press "q" to exit', (30, 470), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    out.write(frame)

    cv2.imshow('Hand Tracking', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
